# Proyecto Final Data Science- CoderHouse

**Equipo**

* David Alejandro Sanabria
* Patricia Mariel Frias
* Marco Vinicio Altamirano F

**Tutor**
* Cristian Velazquez

* INTRODUCCIÓN

En este proyecto se trabajará con un Dataset provisto por Kaggle. En el siguiente link puede encontrarse el dataset: 
https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

El Dataset bajo análisis incluye información de aproximadamente 100 mil órdenes de compra abarcando el período 2016 a 2018 realizadas en múltiples mercados en Brasil. Dicha base contiene los pedidos discriminados por categoría, indicando el precio de los mismos, así como también el costo del transporte. Con ello, es posible calcular el monto de ventas/costos y cantidad de pedidos en forma mensual y anual,y a su vez segregarlos por Ciudad/Estado.

El objetivo de este trabajo es poder implementar un algoritmo predictivo de machine learning que permita conocer de manera precisa las ventas mensuales futuras, así como también responder a una serie de preguntas: ¿Las ventas de una determinada categoría de productos tienen un comportamiento predecible? ¿Es posible estimar las pérdidas de determinadas sucursales? ¿Existe estacionalidad en las ventas? ¿Cómo son esos ciclos, si es que hay? Por supuesto que en la medida que se avance en el trabajo podrán surgir nuevos interrogantes.

El problema específico radica en la posibilidad de contar con una proyección de ventas que permita tener una estimación de la rentabilidad del negocio y así definir estrategias comerciales para lograr una mejor atención al cliente, mejorar la administración de las sucursales y gestionar eficientes campañas de marketing.
 

* Diccionario de Variables

# **Setup**

Las siguientes importaciones están divididas por secciones acordes a su rol en la notebook

In [2]:
# Librerías numéricas, científicas y de ciencia de datos
#===========================================================================
import sys
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # for operations on dataframes
from scipy import stats # for optimization, linear algebra, integration, interpolation
import pandas_profiling as pp # exploratory data analysis EDA
import datetime # for manupulating time and date columns
import warnings
warnings.filterwarnings("ignore")# for warning control


#Visualización de datos
#===========================================================================
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px 


#Importaciones de Sklearn 
#===========================================================================

#Data Preprocessing
from sklearn.cluster import KMeans #for clustering and evaluation
from sklearn.preprocessing import StandardScaler # For handling categorical column and scaling numeric columns


# Toma de datos

In [3]:
# Cargar los datos sin procesar
#===========================================================================
df_clientes = pd.read_csv('../Datasets/olist_customers_dataset.csv')
df_pedidos = pd.read_csv('../Datasets/olist_orders_dataset.csv')
df_productos_en_pedidos = pd.read_csv('../Datasets/olist_order_items_dataset.csv')
df_productos = pd.read_csv('../Datasets/olist_products_dataset.csv')
df_calificaciones = pd.read_csv('../Datasets/olist_order_reviews_dataset.csv')
df_categorias = pd.read_csv("../Datasets/product_category_name_translation.csv")

In [4]:
# Exploración de los campos de la tabla y tipos de datos
#===========================================================================
df_clientes.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [5]:
# Exploración de los campos de la tabla y tipos de datos
#===========================================================================
df_pedidos.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [6]:
# Exploración de los campos de la tabla y tipos de datos
#===========================================================================
df_productos_en_pedidos.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [7]:
# Exploración de los campos de la tabla y tipos de datos
#===========================================================================
df_categorias.dtypes

product_category_name            object
product_category_name_english    object
dtype: object

In [8]:
# Exploración de los campos de la tabla y tipos de datos
#===========================================================================
df_productos.dtypes

product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_photos_qty            float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [9]:
# Exploración de los campos de la tabla y tipos de datos
#===========================================================================
df_calificaciones.dtypes

review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

# Limpieza y preparación de datos

**Data Wrangling y EDA**

In [10]:
# Aplicación de Join para agregar a Productos información del pedido
#===========================================================================
df_productos_comprados = pd.merge(
    df_productos_en_pedidos[['order_id','product_id','price','freight_value']],
    df_productos[['product_id','product_category_name']], on=['product_id'])
df_productos_comprados = df_productos_comprados.merge(df_categorias[['product_category_name']], on=["product_category_name"])

In [11]:
# Aplicación de Join para agregar la informacón del pedido
#===========================================================================
df_pedidos_productos = pd.merge(
    df_productos_comprados, 
    df_pedidos[['order_id','customer_id','order_status','order_purchase_timestamp','order_approved_at',
                'order_delivered_customer_date','order_estimated_delivery_date']], on=['order_id'])


In [12]:
# Aplicación de Join para agregar la información del cliente
#===========================================================================
df_pedidos_clientes = pd.merge(
    df_pedidos_productos,
    df_clientes[['customer_id','customer_city','customer_state']],
    on=['customer_id'])


In [13]:
# Aplicación de Join para agregar las calificaciones
#===========================================================================
df_ventas = pd.merge(df_pedidos_clientes,df_calificaciones[['order_id','review_score']], on=['order_id'])

In [14]:
# Depuración y definición del DtaFrame df_ventas
#===========================================================================
df_ventas = df_ventas.drop(['product_id','customer_id'], axis=1)
df_ventas = df_ventas.rename(columns={'order_id':'Id_Pedidos',
                                      'price':'Precio_Producto',  
                                      'freight_value':'Precio_Flete',
                                      'product_category_name':'Categoria_Producto',
                                      'order_status':'Estatus_Pedido',
                                      'order_purchase_timestamp':'FH_Pedido',
                                      'order_approved_at':'FH_Aprobacion',
                                      'order_delivered_customer_date':'FH_Entrega',
                                      'order_estimated_delivery_date':'F_Estimada_Entrega',
                                      'customer_city':'Ciudad',
                                      'customer_state':'Estado',
                                      'review_score':'Calificacion'})

In [15]:
#Variables del dataset
#===========================================================================
df_ventas.columns

Index(['Id_Pedidos', 'Precio_Producto', 'Precio_Flete', 'Categoria_Producto',
       'Estatus_Pedido', 'FH_Pedido', 'FH_Aprobacion', 'FH_Entrega',
       'F_Estimada_Entrega', 'Ciudad', 'Estado', 'Calificacion'],
      dtype='object')

In [16]:
# Exploración de los primeros 5 registros del dataset
#===========================================================================
df_ventas.head()

,Id_Pedidos,Precio_Producto,Precio_Flete,Categoria_Producto,Estatus_Pedido,FH_Pedido,FH_Aprobacion,FH_Entrega,F_Estimada_Entrega,Ciudad,Estado,Calificacion
0,00010242fe8c5a6d1ba2dd792cb16214,58.9,13.29,cool_stuff,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-20 23:43:48,2017-09-29 00:00:00,campos dos goytacazes,RJ,5
1,130898c0987d1801452a8ed92a670612,55.9,17.96,cool_stuff,delivered,2017-06-28 11:52:20,2017-06-29 02:44:11,2017-07-13 20:39:29,2017-07-26 00:00:00,jatai,GO,5
2,532ed5e14e24ae1f0d735b91524b98b9,64.9,18.33,cool_stuff,delivered,2018-05-18 10:25:53,2018-05-18 12:31:43,2018-06-04 18:34:26,2018-06-07 00:00:00,belo horizonte,MG,4
3,6f8c31653edb8c83e1a739408b5ff750,58.9,16.17,cool_stuff,delivered,2017-08-01 18:38:42,2017-08-01 18:55:08,2017-08-09 21:26:33,2017-08-25 00:00:00,sao jose dos pinhais,PR,5
4,7d19f4ef4d04461989632411b7e588b9,58.9,13.29,cool_stuff,delivered,2017-08-10 21:48:40,2017-08-10 22:05:11,2017-08-24 20:04:21,2017-09-01 00:00:00,conselheiro lafaiete,MG,5


In [17]:
#Información del dataset
#===========================================================================
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110750 entries, 0 to 110749
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Id_Pedidos          110750 non-null  object 
 1   Precio_Producto     110750 non-null  float64
 2   Precio_Flete        110750 non-null  float64
 3   Categoria_Producto  110750 non-null  object 
 4   Estatus_Pedido      110750 non-null  object 
 5   FH_Pedido           110750 non-null  object 
 6   FH_Aprobacion       110736 non-null  object 
 7   FH_Entrega          108457 non-null  object 
 8   F_Estimada_Entrega  110750 non-null  object 
 9   Ciudad              110750 non-null  object 
 10  Estado              110750 non-null  object 
 11  Calificacion        110750 non-null  int64  
dtypes: float64(2), int64(1), object(9)
memory usage: 11.0+ MB


In [18]:
# Conversión del tipo de datos de las variables fecha, de Object a Datetime
#===========================================================================
df_ventas['FH_Pedido'] = df_ventas['FH_Pedido'].astype('datetime64')
df_ventas['FH_Aprobacion'] = df_ventas['FH_Aprobacion'].astype('datetime64')
df_ventas['FH_Entrega'] = df_ventas['FH_Entrega'].astype('datetime64',errors='ignore')
df_ventas['F_Estimada_Entrega'] = df_ventas['F_Estimada_Entrega'].astype('datetime64',errors='ignore')

In [19]:
# Validando existencia de valores nulos
#===========================================================================
df_ventas.isnull().sum().sort_values(ascending=False)

FH_Entrega            2293
FH_Aprobacion           14
Id_Pedidos               0
Precio_Producto          0
Precio_Flete             0
Categoria_Producto       0
Estatus_Pedido           0
FH_Pedido                0
F_Estimada_Entrega       0
Ciudad                   0
Estado                   0
Calificacion             0
dtype: int64

In [20]:
# Reemplazo de valores nulos
#===========================================================================
df_ventas['FH_Aprobacion'].fillna(df_ventas['FH_Pedido'], inplace=True)
df_ventas['FH_Entrega'].fillna(df_ventas['F_Estimada_Entrega'], inplace=True)

In [21]:
# Validando existencia de valores nulos
#===========================================================================
df_ventas.isnull().sum().sort_values(ascending=False)

Id_Pedidos            0
Precio_Producto       0
Precio_Flete          0
Categoria_Producto    0
Estatus_Pedido        0
FH_Pedido             0
FH_Aprobacion         0
FH_Entrega            0
F_Estimada_Entrega    0
Ciudad                0
Estado                0
Calificacion          0
dtype: int64

In [22]:
#Información del dataset
#===========================================================================
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110750 entries, 0 to 110749
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Id_Pedidos          110750 non-null  object        
 1   Precio_Producto     110750 non-null  float64       
 2   Precio_Flete        110750 non-null  float64       
 3   Categoria_Producto  110750 non-null  object        
 4   Estatus_Pedido      110750 non-null  object        
 5   FH_Pedido           110750 non-null  datetime64[ns]
 6   FH_Aprobacion       110750 non-null  datetime64[ns]
 7   FH_Entrega          110750 non-null  datetime64[ns]
 8   F_Estimada_Entrega  110750 non-null  datetime64[ns]
 9   Ciudad              110750 non-null  object        
 10  Estado              110750 non-null  object        
 11  Calificacion        110750 non-null  int64         
dtypes: datetime64[ns](4), float64(2), int64(1), object(5)
memory usage: 11.0+ MB


In [23]:
# Validando existencia de filas duplicadas
#===========================================================================
df_ventas.duplicated(keep='last')

0         False
1         False
2         False
3         False
4         False
          ...  
110745    False
110746    False
110747    False
110748    False
110749    False
Length: 110750, dtype: bool

In [24]:
# Detección de filas duplicadas
#===========================================================================
duplicated_values= df_ventas[df_ventas.duplicated(keep=False)]
duplicated_values

,Id_Pedidos,Precio_Producto,Precio_Flete,Categoria_Producto,Estatus_Pedido,FH_Pedido,FH_Aprobacion,FH_Entrega,F_Estimada_Entrega,Ciudad,Estado,Calificacion
14,45c780334bc32cb77559a65c5f171160,25.00,23.09,cool_stuff,delivered,2017-12-20 19:20:25,2017-12-22 02:39:37,2018-01-03 14:07:22,2018-01-22,para de minas,MG,1
15,45c780334bc32cb77559a65c5f171160,25.00,23.09,cool_stuff,delivered,2017-12-20 19:20:25,2017-12-22 02:39:37,2018-01-03 14:07:22,2018-01-22,para de minas,MG,1
26,dc12f9e8414131e62897902a78e0286e,25.00,17.92,cool_stuff,delivered,2017-12-15 13:41:41,2017-12-15 14:52:47,2017-12-27 19:44:07,2018-01-18,mariana,MG,3
27,dc12f9e8414131e62897902a78e0286e,25.00,17.92,cool_stuff,delivered,2017-12-15 13:41:41,2017-12-15 14:52:47,2017-12-27 19:44:07,2018-01-18,mariana,MG,3
540,ce338e075a36068051ecff6802ab64ef,89.99,8.98,cool_stuff,delivered,2018-05-09 11:31:26,2018-05-09 11:51:17,2018-05-11 19:02:19,2018-05-18,osasco,SP,5
...,...,...,...,...,...,...,...,...,...,...,...,...
110710,9b951f6f81b73f9ee291b9735e6fe993,39.90,25.63,fashion_esporte,delivered,2017-09-28 17:34:38,2017-09-29 02:14:28,2017-10-20 18:02:46,2017-11-03,parauapebas,PA,4
110711,9b951f6f81b73f9ee291b9735e6fe993,39.90,25.63,fashion_esporte,delivered,2017-09-28 17:34:38,2017-09-29 02:14:28,2017-10-20 18:02:46,2017-11-03,parauapebas,PA,4
110712,9b951f6f81b73f9ee291b9735e6fe993,39.90,25.63,fashion_esporte,delivered,2017-09-28 17:34:38,2017-09-29 02:14:28,2017-10-20 18:02:46,2017-11-03,parauapebas,PA,4
110732,3821dc9ac1b25bed42324d0885b0a930,140.00,27.06,la_cuisine,delivered,2018-04-17 21:01:58,2018-04-17 21:15:10,2018-04-23 18:01:57,2018-05-09,sao paulo,SP,1


In [25]:
# Creación de un nuevo dataframe sin filas duplicadas y restablecimiento de índices
#===========================================================================
df_ventas2=df_ventas.drop_duplicates()
df_ventas2.reset_index(drop=True, inplace=True)
df_ventas2

,Id_Pedidos,Precio_Producto,Precio_Flete,Categoria_Producto,Estatus_Pedido,FH_Pedido,FH_Aprobacion,FH_Entrega,F_Estimada_Entrega,Ciudad,Estado,Calificacion
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29,cool_stuff,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-20 23:43:48,2017-09-29,campos dos goytacazes,RJ,5
1,130898c0987d1801452a8ed92a670612,55.90,17.96,cool_stuff,delivered,2017-06-28 11:52:20,2017-06-29 02:44:11,2017-07-13 20:39:29,2017-07-26,jatai,GO,5
2,532ed5e14e24ae1f0d735b91524b98b9,64.90,18.33,cool_stuff,delivered,2018-05-18 10:25:53,2018-05-18 12:31:43,2018-06-04 18:34:26,2018-06-07,belo horizonte,MG,4
3,6f8c31653edb8c83e1a739408b5ff750,58.90,16.17,cool_stuff,delivered,2017-08-01 18:38:42,2017-08-01 18:55:08,2017-08-09 21:26:33,2017-08-25,sao jose dos pinhais,PR,5
4,7d19f4ef4d04461989632411b7e588b9,58.90,13.29,cool_stuff,delivered,2017-08-10 21:48:40,2017-08-10 22:05:11,2017-08-24 20:04:21,2017-09-01,conselheiro lafaiete,MG,5
...,...,...,...,...,...,...,...,...,...,...,...,...
99579,3da4b0f0c89bb8fee8a810459e641414,110.00,8.96,fashion_roupa_infanto_juvenil,delivered,2017-06-08 16:46:33,2017-06-08 18:20:19,2017-06-13 13:23:01,2017-06-23,sao paulo,SP,5
99580,9fa3a005f89a187501fa1bd63d6125fb,39.99,14.10,fashion_roupa_infanto_juvenil,delivered,2017-10-08 23:51:49,2017-10-09 00:17:50,2017-10-16 21:59:33,2017-11-08,sao paulo,SP,5
99581,cab08f0417a702d0f4782ec02356a4ac,89.99,17.07,fashion_roupa_infanto_juvenil,delivered,2017-09-30 18:54:53,2017-10-03 04:28:12,2017-10-30 12:37:42,2017-11-01,macaubas,BA,5
99582,cb53f0b6e8f9082a9f5a113765dd1236,89.99,7.49,fashion_roupa_infanto_juvenil,delivered,2018-06-03 22:34:58,2018-06-03 22:50:09,2018-06-05 21:52:02,2018-06-28,sao paulo,SP,5


In [26]:
#Resumen rápido de las estadísticas de datos
#===========================================================================
df_ventas2.describe()

,Precio_Producto,Precio_Flete,Calificacion
count,99584.000000,99584.000000,99584.000000
mean,124.744862,20.167933,4.081569
std,187.671684,15.918764,1.348826
min,0.850000,0.000000,1.000000
25%,40.990000,13.180000,4.000000
50%,79.000000,16.360000,5.000000
75%,139.200000,21.300000,5.000000
max,6735.000000,409.680000,5.000000


In [32]:
#Resumen del dataset
#===========================================================================
profile = pp.ProfileReport(df_ventas2)

profile


In [ ]:
#Identificación de posibles estados del pedido
#===========================================================================
df_ventas2["Estatus_Pedido"].value_counts()

In [ ]:
# Identificación del total de órdenes de compra no concretadas
#===========================================================================
ordenes_no_entregadas = df_ventas2[df_ventas2["Estatus_Pedido"] != 'delivered']
ordenes_no_entregadas ["Estatus_Pedido"].value_counts().sum()

In [ ]:
# Creación de Dataframe que sólo incluye pedidos entregados
#===========================================================================
Ventas= df_ventas2[df_ventas2['Estatus_Pedido'] =='delivered']
Ventas

# Análisis de datos

**Análisis Univariado**

In [ ]:
# Analizar el volumen de cada estado de pedido.

orderc = df_ventas2['Estatus_Pedido'].value_counts(ascending=False)

plt.figure(figsize=(10,5))

palette = sns.color_palette('Greens_d', len(orderc))
ax = sns.barplot(orderc.index, orderc.values, alpha=0.8 , palette=np.array(palette[::-1]))

plt.title('Recuento de estado de pedidos', fontsize=16)
plt.ylabel('Cantidad', fontsize=12)
plt.xlabel('Estado_Pedido', fontsize=12)
plt.ylim(0,120000) 
ax.bar_label(ax.containers[0])

plt.show()

**OBSERVACIÓN**: En función de lo observado en la gráfica y para nuestro posterior análisis, se considerarán como ventas concretadas solamente aquellas órdenes de compra efectivamente entregadas al cliente.

**Análisis Bivariado**

**Análisis Multivariado**

In [ ]:
sns.pairplot(df_ventas2, corner=True)

In [ ]:
plt.figure(figsize=(20,20))

sns.heatmap(df_ventas2.corr(), annot=True, vmin=-1, vmax=1, cmap="bwr_r", linewidths=.4)

plt.show()
